In [ ]:
import aero_vloc as avl

from pathlib import Path

Downloading a satellite map from Google Maps

In [ ]:
map_downloader = avl.MapDownloader(north_west_lat=46.066412, north_west_lon=38.121383, 
                                   south_east_lat=45.999099, south_east_lon=38.250987,
                                   zoom=17, api_key="YOUR API KEY HERE", 
                                   folder_to_save=Path('tests/test_data/map'))
map_downloader.download_map

Specify paths to satellite map and aerial queries.
You can also specify arbitrary zoom level and overlap level between frames.

In [ ]:
linear_referencer = avl.LinearReferencer()

satellite_map = avl.Map(Path("tests/test_data/map/map_metadata.txt"), zoom=1.5, overlap_level=0.25, geo_referencer=linear_referencer)
drone_images = avl.UAVSeq(Path("tests/test_data/queries/queries.txt"))

Create a VPR system as well as a re-ranking method. EigenPlaces and SuperGlue are used in the example

In [ ]:
eigen_places = avl.EigenPlaces('ResNet101', fc_output_dim=2048)
super_glue = avl.SuperGlue("weights/superglue_outdoor.pth", resize=800)
faiss_searcher = avl.FaissSearcher()
retrieval_system = avl.RetrievalSystem(eigen_places, satellite_map, super_glue, faiss_searcher)

homography_estimator = avl.HomographyEstimator()
localization_pipeline = avl.LocalizationPipeline(retrieval_system, homography_estimator)

Now you can run provided metrics

In [ ]:
recall_value = avl.reference_recall(drone_images, localization_pipeline, k_closest=50, threshold=100)
print(recall_value)